In [ ]:
from sklearn.neighbors import NearestNeighbors

# 1. بناء مصفوفة التفاعل: المستخدم × المرشد
interaction_matrix = df_interactions.pivot_table(
    index='user_id',
    columns='guide_id',
    values='interest_match_score',
    fill_value=0
)

# 2. تدريب نموذج KNN
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(interaction_matrix)

# 3. اختيار مستخدم لتجربة التوصيات (مثلاً أول مستخدم)
sample_user_index = 0
sample_user_id = interaction_matrix.index[sample_user_index]

# 4. استخراج أقرب جيران للمستخدم
distances, indices = knn.kneighbors(
    interaction_matrix.iloc[sample_user_index].values.reshape(1, -1),
    n_neighbors=6  # 1 نفسه + 5 جيران
)

# 5. استخراج user_ids للجيران (بدون المستخدم نفسه)
neighbor_ids = interaction_matrix.index[indices.flatten()[1:]]

# 6. المرشدين الذين تفاعل معهم الجيران (لكن المستخدم المستهدف لم يتفاعل معهم بعد)
neighbor_interactions = df_interactions[df_interactions['user_id'].isin(neighbor_ids)]
user_guides = df_interactions[df_interactions['user_id'] == sample_user_id]['guide_id'].unique()
candidate_guides = neighbor_interactions[~neighbor_interactions['guide_id'].isin(user_guides)]

# 7. تجميع متوسط score للمرشدين بناء على تفاعل الجيران
top_recommended = (
    candidate_guides.groupby('guide_id')['interest_match_score']
    .mean()
    .reset_index()
    .sort_values(by='interest_match_score', ascending=False)
    .head(5)
)

# 8. ربط أسماء المستخدم والمرشد + الاهتمامات والتاغز + المناطق
# --- بيانات المستخدم
user_name = df_users[df_users['id'] == sample_user_id]['name'].values[0]
user_region = df_users[df_users['id'] == sample_user_id]['region'].values[0]
user_interests = "، ".join(df_interests[df_interests['user_id'] == sample_user_id]['interest'].tolist())

# --- دمج مع أسماء المرشدين
top_recommended = top_recommended.merge(df_guides[['id', 'name', 'region']], left_on='guide_id', right_on='id')
top_recommended = top_recommended.rename(columns={
    'name': 'guide_name',
    'region': 'guide_region',
    'interest_match_score': 'similarity'
})

# --- دمج مع التاغز
guide_tags = df_tags.groupby("guide_id")["tag"].apply(lambda x: "، ".join(x)).reset_index()
top_recommended = top_recommended.merge(guide_tags, left_on='guide_id', right_on='guide_id', how='left')

# 9. بناء جدول التوصيات النهائي
df_knn_cleaned = top_recommended[[
    'guide_name', 'tag', 'guide_region', 'similarity'
]].copy()

df_knn_cleaned.insert(0, "user_name", user_name)
df_knn_cleaned.insert(1, "user_interests", user_interests)
df_knn_cleaned.insert(2, "user_region", user_region)

# 10. إعادة التسمية النهائية بالعربي
df_knn_cleaned = df_knn_cleaned.rename(columns={
    "user_name": "المستخدم",
    "user_interests": "اهتماماته",
    "user_region": "منطقته",
    "guide_name": "المرشد المقترح",
    "tag": "اهتمامات المرشد",
    "guide_region": "منطقته (المرشد)",
    "similarity": "نسبة التطابق"
})

# 11. عرض النتائج
display(df_knn_cleaned)
